In [8]:
# Import required libraries
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import numpy as np
from os import urandom
import sympy as sp
from IPython.display import display
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

In [9]:
# AES key
key = os.urandom(16)  # 128-bit key

In [10]:
# AES block size
block_size_bytes = 16

In [11]:
# Plaintext (arbitrary length)
plaintext = b'A un amigo perdido. Ya no me debes recordar en el pais extranjero. Tambien a mi se me escapa tu imagen lejana.'  

In [12]:
# 1. Apply PKCS7 padding
padder = padding.PKCS7(block_size_bytes * 8).padder()
padded_plaintext = padder.update(plaintext) + padder.finalize()

### 2. Create a new AES-ECB encryptor and encrypt the plaintext

In this step, we take the **padded plaintext** and encrypt it using **AES in ECB mode**.

```python
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
print("Ciphertext (hex):", ciphertext.hex())


In [13]:
# 2. Create a new AES-ECB encryptor
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
print("Ciphertext (hex):", ciphertext.hex())

Ciphertext (hex): d84efda36acdb55ccef2a054bbdeca597eafcebd5bd8e5bdda2d7c68f845db638f3f6bb8bf4c94fe789db0b8c5657fe71421428bb5907f2be517a967ea56ebeebca667b7efc3e8106a9656f4aa255c897d8092a3547d6a2b1b10b3efd1d6698d793558fb5f7993c61ed7517f9071cf49


### Step-by-step explanation

**`Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())`**  
- Creates a cipher object using AES with the given key.  
- `modes.ECB()` specifies the Electronic Codebook (ECB) mode, the simplest AES mode.  
- `backend=default_backend()` tells the library to use the system’s default cryptographic backend (usually OpenSSL).

**`encryptor = cipher.encryptor()`**  
- Creates a one-time encryption context.  
- Each encryptor can only be used once for a single encryption operation.

**`encryptor.update(padded_plaintext) + encryptor.finalize()`**  
- `update(padded_plaintext)` processes the plaintext in 16-byte blocks.  
- `finalize()` completes the encryption and returns any remaining bytes.  
- The result is the ciphertext, which is exactly the same length as the padded plaintext.

**`print("Ciphertext (hex):", ciphertext.hex())`**  
- Prints the ciphertext as a hexadecimal string so it is human-readable.

---

### Notes for students

- ECB mode encrypts each 16-byte block independently, so identical blocks in the plaintext will produce identical ciphertext blocks.  
- ECB is not recommended for real applications, but it is excellent for teaching the basic encryption process.  
- Padding ensures the plaintext length is a multiple of the AES block size (16 bytes), which is required by ECB mode.


In [14]:
# 3. Decrypt
decryptor = cipher.decryptor()
decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()

### Step 3: Decrypt the ciphertext

**`decryptor = cipher.decryptor()`**  
- Creates a new decryption context from the same cipher object.  
- Each decryptor can only be used once for a single decryption operation.  

**`decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()`**  
- `update(ciphertext)` processes the ciphertext in 16-byte blocks and decrypts it.  
- `finalize()` completes the decryption and returns any remaining bytes.  
- The result is the **padded plaintext**, which still includes the PKCS#7 padding added during encryption.

---

**Notes for students:**

- The decrypted output is not yet the original plaintext; it still contains padding.  
- AES-ECB decrypts each block independently, matching the way encryption was done.  
- A new decryptor must be created each time you perform decryption; you cannot reuse the same object for multiple operations.


In [15]:
# 4. Remove padding
unpadder = padding.PKCS7(block_size_bytes * 8).unpadder()
recovered = unpadder.update(decrypted_padded) + unpadder.finalize()
print("Recovered plaintext:", recovered)


Recovered plaintext: b'A un amigo perdido. Ya no me debes recordar en el pais extranjero. Tambien a mi se me escapa tu imagen lejana.'


### Step 4: Remove padding

**`unpadder = padding.PKCS7(block_size_bytes * 8).unpadder()`**  
- Creates an **unpadding context** for AES.  
- AES block size = 128 bits (16 bytes), so we multiply by 8 to get bits for PKCS#7.

**`recovered = unpadder.update(decrypted_padded) + unpadder.finalize()`**  
- `update(decrypted_padded)` removes the padding bytes from the decrypted data.  
- `finalize()` completes the unpadding process.  
- The result is the **original plaintext**, exactly as it was before encryption.

**`print("Recovered plaintext:", recovered)`**  
- Displays the recovered plaintext to verify that encryption and decryption worked correctly.

---

**Notes for students:**

- Padding is necessary because AES operates on fixed 16-byte blocks.  
- PKCS#7 padding adds extra bytes to make the plaintext a multiple of the block size.  
- Removing the padding restores the original message.  
- This step is essential when encrypting plaintexts of arbitrary length.
